Central Limit Theorem: in this lab we demonstrate the Central Limit Theorem by sampling from a given distribution.

Central Limit Theorem (CLT) - see Probability Theory course for more versions.
Let $(X_n)_{n\geq 1}$ be a sequence of iid random variables such that
$\mu=\mathbb{E}\left(X_n\right)$ and $\sigma^2=V\left(X_n\right)>0 \forall n \geq 1$. Then

$$
\frac{\left(X_1+\ldots+X_n\right)-n \mu}{\sigma \sqrt{n}} \xrightarrow{d} Z, \quad \text { where } Z \sim N(0,1),
$$

this means

$$
\lim _{n \rightarrow \infty} \mathbb{P}\left(\frac{\left(X_1+\ldots+X_n\right)-n \mu}{\sigma \sqrt{n}} \leq x\right)=\frac{1}{\sqrt{2 \pi}} \int_{-\infty}^x e^{-\frac{t^2}{2}} d t=F_{N(0,1)}(x)
$$

$\forall x \in \mathbb{R}$ where $F_{N(0,1)}$ denotes the cumulative distribution function of the standard normal distribution $N(0,1)$.

**Exercise 1:** Consider n numbers $x_1, x_2, \ldots, x_n$ randomly selected from the uniform distribution $U(0,1)$. Let $$\bar{x}:= \frac{1}{n}\sum_{i=1}^n x_i$$ be the sample mean. When $n=100$ find the probability that $\bar{x} > 0.55$.

**Exercise 2:** Suppose a sample $$x_1, ..., x_n$$ is modeled by a Poisson distribution with parameter λ, where

$$f_X(x; λ) = (λ^x / x!) * exp(-λ)$$,   for x = 0, 1, 2, ... and λ > 0.

Estimate λ using the maximum likelihood method (MLE) and compare it with the method of moments (MM) estimator, using the following data:

Observed frequencies for domestic accidents (n = 647):

Number of accidents: 0 1 2 3 4 5

Frequency:           447 132 42 21 3 2 (respectively)

**Exercise 3:**

Estimate the unknown parameter theta from a sample:

3, 3, 3, 3, 3, 7, 7, 7

This sample is drawn from a discrete distribution with the probability mass function:

$$P(X = 3) = \theta$$
$$P(X = 7) = 1 - \theta$$

Compute two estimators of theta:

1. Method of moments estimator
2. Maximum likelihood estimator

**Exercise 4:** Suppose we have a sample $X_1, \dots, X_n$ drawn from a **Beta distribution** with unknown parameters $\alpha$ and $\beta$:

$$
f_X(x; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1-x)^{\beta-1}, \quad 0 < x < 1
$$

**Tasks:**

1. Compute the **Method of Moments (MM) estimates** for $\alpha$ and $\beta$ using the sample mean and variance.
2. Compute the **Maximum Likelihood Estimates (MLE)** numerically using `scipy.optimize` (MLE for Beta has no closed form).

- Analyse the code below :)

Estimating Parameters of a Beta Distribution

### 1. Method of Moments

For a $Beta(\alpha, \beta)$ distribution:

$$
\mathbb{E}[X] = \frac{\alpha}{\alpha + \beta}, \quad
\text{Var}(X) = \frac{\alpha \beta}{(\alpha + \beta)^2 (\alpha + \beta + 1)}
$$

Let $\bar{x}$ and $s^2$ denote the sample mean and sample variance. The method of moments sets (check):

$$
\bar{x} = \frac{\alpha}{\alpha + \beta}, \quad
s^2 = \frac{\alpha \beta}{(\alpha + \beta)^2 (\alpha + \beta + 1)}
$$

Solving for $\alpha$ and $\beta$ (check) gives:

$$
\text{common} = \frac{\bar{x}(1-\bar{x})}{s^2} - 1, \quad
\alpha_{\text{MM}} = \bar{x} \cdot \text{common}, \quad
\beta_{\text{MM}} = (1-\bar{x}) \cdot \text{common}
$$

This provides a quick analytical estimate.

---

### 2. Maximum Likelihood Estimation

The log-likelihood of a sample $\{x_i\}_{i=1}^n$ from a Beta(\(\alpha, \beta\)) distribution is:

$$
\ell(\alpha, \beta) = \sum_{i=1}^n \Big[ (\alpha-1)\ln x_i + (\beta-1)\ln(1-x_i) - \ln B(\alpha, \beta) \Big]
$$

where $B(\alpha, \beta) = \frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha+\beta)}$ is the Beta function.  

Since the MLE equations cannot be solved analytically, we **numerically maximize** the log-likelihood using Python (`scipy.optimize.minimize`), typically starting from the Method of Moments estimates as initial guesses.




In [ ]:
import numpy as np
from scipy.special import beta as beta_func, digamma
from scipy.optimize import minimize

# Example sample from a Beta distribution (unknown alpha, beta)
np.random.seed(42)
sample = np.random.beta(a=2.5, b=5.0, size=1000)

# -----------------------------
# Method of Moments Estimator
# -----------------------------
x_bar = np.mean(sample)
s2 = np.var(sample, ddof=1)  # sample variance

# Solve MM equations:
#   alpha_hat / (alpha_hat + beta_hat) = x_bar
#   alpha_hat * beta_hat / ((alpha_hat + beta_hat)^2 * (alpha_hat + beta_hat + 1)) = s2
common = x_bar * (1 - x_bar) / s2 - 1
alpha_mm = x_bar * common
beta_mm = (1 - x_bar) * common

print(f"Method of Moments estimates: alpha = {alpha_mm:.4f}, beta = {beta_mm:.4f}")

# -----------------------------
# Maximum Likelihood Estimator
# -----------------------------
def neg_log_likelihood(params):
    alpha, beta = params
    if alpha <= 0 or beta <= 0:
        return np.inf  # parameters must be positive
    return -np.sum((alpha-1)*np.log(sample) + (beta-1)*np.log(1-sample) - np.log(beta_func(alpha, beta)))

# Initial guess: Method of Moments estimates
initial_guess = [alpha_mm, beta_mm]

result = minimize(neg_log_likelihood, initial_guess, method='L-BFGS-B', bounds=[(1e-6, None), (1e-6, None)])
alpha_mle, beta_mle = result.x

print(f"Maximum Likelihood estimates: alpha = {alpha_mle:.4f}, beta = {beta_mle:.4f}")
